In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

!pip install hazm

import numpy as np
import cv2 as cv
import pandas as pd
from hazm import *
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report
import math
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import precision_score, accuracy_score, recall_score
import matplotlib.pyplot as plt
from string import punctuation
import random

     |████████████████████████████████| 317kB 7.6MB/s 
     |████████████████████████████████| 1.4MB 12.0MB/s 
     |████████████████████████████████| 235kB 15.7MB/s 
  Created wheel for nltk: filename=nltk-3.3-cp36-none-any.whl size=1394469 sha256=f7eeafbf4e19d927ec34c51e32fb377b8d0c486ffc80acf48aaa1df61a320b96
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp36-cp36m-linux_x86_64.whl size=154538 sha256=c4c233f78889380e6fc7a2da9eacfbf08e0f6ca2cc5e26827a991999751df62e
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
Successfully built nltk libwapiti
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [3]:
url = 'drive/My Drive/train.csv'
df = pd.read_csv(url, engine='python',encoding='utf-8', error_bad_lines=False)
test_url = 'drive/My Drive/test.csv'
df_test = pd.read_csv(test_url, engine='python',encoding='utf-8', error_bad_lines=False)

In [4]:
df.head()

,Unnamed: 0,Text,Category
0,0,\nخبرنامه دانشگاه علم و صنعت ايران \nشماره ياز...,Science and Culture
1,1,\nتا پايان سال 1378 دهها زمين فوتبال و \nسالن ...,Sport
2,2,\nانجمن توليدكنندگان تجهيزات صنعت نفت تشكيل شد...,Economy
3,3,\nكرتين براي سومين بار نخست وزير كانادا \nشد \...,Miscellaneous.World News
4,4,\nخداحافظ رفقا \nنمايندگان اروپاي شرقي در جام ...,Sport


In [5]:
df_test.head()

,Id,Text
0,0,\nهفت اقليم \nآلودگي هوا پكن را تهديد مي كند \...
1,1,\nگل و گياه زعفران زينتي \nنام علمي: Crocus ba...
2,2,\nيادداشت \nقانون بودجه و صنايع كوچك \nدر شمار...
3,3,\nدر سالروز ميلاد حضرت مهدي \nهمايش ادبي دانش ...
4,4,\nاز IRA تا فارك \n بوگوتا، پايتخت پرهرج ومرج ...


In [6]:
train_len = df.shape[0]
print(train_len)

150096


In [7]:
test_len = df_test.shape[0]
print(test_len)

16678


# *Preprocessing*

In [56]:
normalizer = Normalizer()
lemmatizer = Lemmatizer()
stemmer = Stemmer()

stop_words= []
with open("drive/My Drive/stopwords.txt" ,encoding = 'utf-8' ) as f:
    stop_words = f.read().splitlines()
stopwords = stop_words [1:]
print(stopwords)


def preprocessing(text):
    # text = normalizer.normalize(text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if not word in stopwords]
    cleared_text = []
    for word in tokens:
          # word = normalizer.normalize(word)
          word = re.sub(r'\d+', '', word)
          word = re.sub(r'\s*[A-Za-z]+\b', '', word)
          cleared_text.append(word)
    return cleared_text

def clean_text(text):
  output = []
  for i in range(len(text)):
    if(i % 10000 == 0):
      print("iteration # {}".format(i))
    clean_txt = preprocessing(text[i])
    output.append(clean_txt)
  return output

['"', '#', '%', '\\', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '\\\\', '//', '?', '@', '\\', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '’', '«', '»', '”', '“', '′', '‘', '،', '\\\\', 'و', 'در', 'به', 'از', 'که', 'این', 'را', 'با', 'است', 'برای', 'آن', 'یک', 'خود', 'تا', 'کرد', 'بر', 'هم', 'نیز', 'گفت', 'می\u200cشود', 'وی', 'شد', 'دارد', 'ما', 'اما', 'یا', 'شده', 'باید', 'هر', 'آنها', 'بود', 'او', 'دیگر', 'دو', 'مورد', 'می\u200cکند', 'شود', 'کند', 'وجود', 'بین', 'پیش', 'شده_است', 'پس', 'نظر', 'اگر', 'همه', 'یکی', 'حال', 'هستند', 'من', 'کنند', 'گردد', 'حتما', 'سلام', 'دلم', 'دیدن', 'بزنید', 'صحبتهای', 'عنوان', 'معناست', 'خبرهای', 'شب', 'نیست', 'باشد', 'چه', 'عده\u200cای', 'علیه\u200cالسلام', 'باشیم', 'فعلی', 'صد', 'ان', 'انها', 'جناب', 'جناب\u200cاقای', 'میشود', 'عده', 'بنا', 'دکتر', 'عرض', 'بایستی', 'نمایند', 'دست', 'بیش', 'روز\u200cگذشته', 'اشاره', 'میگن', 'کنه', 'مارو', 'اونم', 'اینم', 'فک', 'کردین

In [57]:
preprocessed_text_train = clean_text(df['Text'])

iteration # 0
iteration # 10000
iteration # 20000
iteration # 30000
iteration # 40000
iteration # 50000
iteration # 60000
iteration # 70000
iteration # 80000
iteration # 90000
iteration # 100000
iteration # 110000
iteration # 120000
iteration # 130000
iteration # 140000
iteration # 150000


In [58]:
preprocessed_text_test = clean_text(df_test['Text'])

iteration # 0
iteration # 10000


# Word Embedding

In [59]:
def dummy_fun(doc):
    return doc
def tfidf(data, ma = 0.6, mi = 0.0001):
    tfidf_vectorize = TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None,
    max_df = ma,
    min_df = mi) 
    tfidf_data = tfidf_vectorize.fit_transform(data)
    return tfidf_data

In [60]:
word_embedded_text_train = tfidf(preprocessed_text_train)
print(word_embedded_text_train.shape)

(150096, 40382)


In [61]:
word_embedded_text_test = tfidf(preprocessed_text_test)
print(word_embedded_text_test.shape)

(16678, 47958)


In [62]:
merge_train_test = preprocessed_text_train
for i in range(0 , len(preprocessed_text_test)):
    merge_train_test.append(preprocessed_text_test[i])

In [63]:
merge_word_embedded_text = tfidf(merge_train_test)

# Split Data

In [64]:
import sklearn
def train_test_spiliting(X,Y):
  x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X, Y, test_size = 0.05, random_state = 42)
  print("X_train lenght: {}".format(x_train.shape[0]))
  print("X_test lenght: {}".format(x_test.shape[0]))
  return x_train, x_test, y_train, y_test 

In [65]:
x_train, x_test, y_train, y_test = train_test_spiliting(merge_word_embedded_text[0:150096] , df['Category'])

X_train lenght: 142591
X_test lenght: 7505


# Model

In [66]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import ReduceLROnPlateau
def test_logestic(x_train, x_test, y_train, y_test):
    model = Pipeline([ ('clf', LogisticRegression(n_jobs = 1 , C = 1e5))])
    classifier = model.fit(x_train,y_train)
    predictions = classifier.predict(x_test)
    print(classification_report(y_test, predictions))
    a = accuracy_score(y_test, predictions)
    p = precision_score(y_test, predictions, average = 'weighted')
    r = recall_score(y_test, predictions, average = 'weighted')
    return a , p, r ,classifier

In [67]:
def train(x_train, x_test, y_train, y_test):
  accuracy, precision, recall , classifier = test_logestic(x_train, x_test, y_train, y_test)
  print("model accuracy {}".format(accuracy))
  print("model precision {}".format(precision))
  print("model recall {}".format(recall))
  return classifier

In [68]:
classifier = train(x_train, x_test, y_train, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                                      precision    recall  f1-score   support

                                                             Economy       0.81      0.87      0.84       874
                                                 Economy.Agriculture       0.91      0.43      0.59        23
                                             Economy.Bank and Bourse       0.60      0.69      0.64        51
                                                    Economy.Commerce       0.67      0.14      0.24        14
                                   Economy.Dwelling and Construction       0.00      0.00      0.00         3
                                                    Economy.Industry       0.36      0.20      0.26        40
                                                         Economy.Oil       0.75      0.75      0.75        48
                                                  Literature and Art       0.62      0.60      0.61       238
         

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Test Prediction


In [69]:
def prediction(word_embedded_text_test):
  predictions = classifier.predict(word_embedded_text_test)
  return predictions

In [70]:
predictions = prediction(merge_word_embedded_text[150096:])

In [71]:
import csv
def save_prediction(predict,df_test):
  myfile = open("result.csv", "w")
  writer = csv.writer(myfile)
  writer.writerow(["Id", "Category"])

  for i in range(len(df_test)):
    writer.writerow([df_test['Id'][i], predict[i]])

  myfile.close()

In [72]:
save_prediction(predictions,df_test)